# Prepare data

In [1]:
from script.func_split_data import split_data

In [2]:
split_df = split_data(data_path='Cosmenet_product_20231018', data_csv='datas_20231018.csv')
split_df.split_data()
split_df.report_train_test_split()
print()
split_df.report_train_test_val_split()

amount of all data : 60196
amount of all class : 4178
amount of data 2-8 img : 1548
amount of 2-8 img class : 278
amount of data more 8 img : 58631
amount of more 8 img class : 3883
amount of data & class only one : 17

amount of train split : 38474
amount of train split class : 3184
amount of test split : 9620
amount of test split class : 3184
amount of train val : 8430
amount of train val class : 699
amount of test val : 2107
amount of test val class : 699
amount of train val mix : 9204
amount of train val mix class : 977
amount of test val mix : 2881
amount of test val mix class : 977


In [3]:
df_train_split, df_test_split = split_df.get_train_test()
df_train_val_mix, df_test_val_mix = split_df.get_validate()

In [4]:
df_train_split.head(1)

,file_names,labels,images_path
0,11596_2.jpg,11596,/app/nfs_clientshare/Datasets/Cosmenet_product...


In [5]:
df = split_df.get_dict()

# extract and put to elasticsearch

In [6]:
import torch
from transformers import ViTImageProcessor, ViTModel

from script.func_extract_feature import select_transformers_model, pipeline_transformer
from script.tool import ROOT_NFS_TEST

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, preprocess = select_transformers_model(ViTModel, ViTImageProcessor, 
                                              pretrain=ROOT_NFS_TEST / 'weights/vit_gg_lr2e-05_eu_9ep_0_95099acc')
model.load_state_dict(torch.load(ROOT_NFS_TEST / 'weights/temp_epoch/vitgg_lr2e05_ep5_loss0.0.pth')['model_state_dict'])
vit_gg_trained_lr2e_05_pipe = pipeline_transformer(layer="last_hidden_state", row=0, device=device)
vit_gg_trained_lr2e_05_pipe.selct_model(model, preprocess)
vit_gg_trained_lr2e_05_pipe.report_test()

runtime : 332.99803733825684 ms
outputs layers : odict_keys(['last_hidden_state', 'pooler_output'])
shape last_hidden_state : torch.Size([1, 197, 768])
shape pooler_output : torch.Size([1, 768])


In [8]:
from script.func_query_score import extract_to_es

In [17]:
ext_ep3_crop = extract_to_es('vitgg_lr2e05_ep5_loss0.0', url='http://192.168.1.97:9200')
ext_ep3_crop.check_index_exist()
ext_ep3_crop.put_all_tag(vit_gg_trained_lr2e_05_pipe, df, replace=False, crop=False)

index vitgg_lr2e05_ep5_loss0.0 already exists


  0%|          | 0/38474 [00:00<?, ?it/s]

43968
45422
40562
41114
42125
40562
48387
42125
47701
48387
48451
47701
46973
48451
47000
43678
9472
47000
41958
9472
47111
41958
41644
47111
45430
43387
43068
41644
41817
45430
40127
46275
45798
36896
46945
43068
40537
41817
41301
40127
44802
45798
47209
46945
44407
40537
47080
41301
42528
44802
46426
47209
44732
44407
41617
47080
36658
42528
41991
39748
46020
44732
13132
41617
40358
36658
39483
41991
44987
46020
7777
13132
46404
40358
34907
39483
39788
46404
47009
34907
44063
39788
43414
47009
44010
43414
41931
44010
27782
41931
13943
27782
41630
13943
38968
41630
46278
38968
39495
46278
47006
39495
41290
47006
31347
41290
42528
31347
45298
42528
39623
45298
20249
39623
41242
46868
41630
41242
43417
41630
49469
43417
39538
44508
39976
39538
40393
43982
45877
39976
23474
40511
43927
40393
48695
45877
43514
43038
41307
1079
41760
23474
put tag train_split success


  0%|          | 0/9620 [00:00<?, ?it/s]

put tag test_split success


  0%|          | 0/9204 [00:00<?, ?it/s]

# Extract Elasticnet b1

In [39]:
from script.func_extract_feature import select_timm_model, pipeline_timm

In [40]:
model, preprocess = select_timm_model('efficientnet_b1', num_classes=0, pretrain=True)
eff_pipe = pipeline_timm(device=device)
eff_pipe.selct_model(model, preprocess)
eff_pipe.report_test()

runtime : 179.80599403381348 ms
Output shape at layer : torch.Size([1, 1280])


In [41]:
eff = extract_to_es('efficientnet_b1')
eff.check_index_exist(dims=1280)
eff.put_all_tag(eff_pipe, df, replace=False, crop=False)

index efficientnet_b1 already exists


  0%|          | 0/38474 [00:00<?, ?it/s]

put tag train_split success


  0%|          | 0/9620 [00:00<?, ?it/s]

put tag test_split success


  0%|          | 0/9204 [00:00<?, ?it/s]

/tmp/ipykernel_330647/2751088454.py:35: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index=self.name_index, id=tag+"_"+str(n), body=data)


put tag train_val success


  0%|          | 0/2881 [00:00<?, ?it/s]

put tag test_val success


# measure_score

In [ ]:
from script.func_query_score import measure_score, report_image

In [ ]:
url = "192.168.1.97:9200"

In [ ]:
measure_15k = measure_score("efficientnet_b1", url=url)
result_15k = measure_15k.report_acc()

## Show image

In [ ]:
report = report_image(name_index="vitgg_lr2e05_ep3_loss0.0", url=url)
rand_id_split, rand_id_val = report.show_report(num_product=5, top_n=5, collapse=True)